# Filtering & Sorting Data in SQL

#### Databases are commonly used for persostent data storage, and therefore it is common to add or remove rows as new data is created (e.g someone places an order) or destroyed (e.g a product is discontinued). This may be performed automatically via an application's database connection; we wil use database connections later in this notebook. However, in the mean time we wil load in a larger version of the first data set (Working with SQL) from file for analysis 

In [56]:
import pandas as pd
import sqlite3


In [57]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [58]:
# create a database connection
%sql sqlite:///testdb.sqlite

'Connected: @testdb.sqlite'

In [61]:
conn = sqlite3.connect('testdb.sqlite')
customers = pd.read_csv('data/customers.csv')
products = pd.read_csv('data/products.csv')
orders = pd.read_csv('data/orders.csv')

In [66]:
products.head()

,id,price
0,0,21.78
1,1,18.48
2,2,24.17
3,3,29.71
4,4,20.12


In [65]:
orders.head()

   id  price
0   0  21.78
1   1  18.48
2   2  24.17
3   3  29.71
4   4  20.12


,id,customer_id,product_id,delivery_country
0,0,1866,686,India
1,1,1921,617,China
2,1,1921,54,Armenia
3,1,1921,566,China
4,1,1921,128,Lebanon


### We will use pandas ***to_sql*** method to create table in our database

In [59]:
customers.to_sql("customers", conn, index=False, if_exists = 'replace')
products.to_sql("products", conn, index = False, if_exists = 'replace')
orders.to_sql("orders", conn, index=False, if_exists = 'replace')

In [74]:
%%sql
SELECT * FROM orders
    LIMIT 5;

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,customer_id,product_id,delivery_country
0,1866,686,India
1,1921,617,China
1,1921,54,Armenia
1,1921,566,China
1,1921,128,Lebanon


In [75]:
%%sql
SELECT * FROM customers
    LIMIT 5;

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,name,billing_country
1,Serenity Ladner,Canada
2,Lucy Baldwin,Canada
5,Skylar Coffman,USA
6,Riley Kinsey,USA
7,Heidi Spurlock,Canada


In [76]:
%%sql
SELECT * FROM products
    LIMIT 5;

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,price
0,21.78
1,18.48
2,24.17
3,29.71
4,20.12


### Filtering and sorting data
#### Filtering is principally accomplished using the $WHERE$ command

In [79]:
%%sql
SELECT id, delivery_country FROM orders
WHERE delivery_country = 'India'
LIMIT 5;

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,delivery_country
0,India
60,India
60,India
60,India
64,India


#### Note the single equals sign *$=$* used in the conditional $WHERE$ keyword. This is different from python where double *==* is used.

#### In SQL, we do not assign values to variables as in python. So there is no ambiguity or neccessity for a single equals sign to represnt assignment rather than eqaultiy

###  *SQL  is  a  query  language*. 
#### It is used for performing trnasformation and aggregation of data present in databases. It is not meant for writing applications.

### We can perform other conditional filtering with symbols such as $  <,  >,  != , <=, >= $

In [83]:
%%sql

SELECT * FROM products
WHERE price != 20
LIMIT 5;

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,price
0,21.78
1,18.48
2,24.17
3,29.71
4,20.12


In [93]:
%%sql

SELECT * FROM products
WHERE price > 20
LIMIT 5;

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,price
0,21.78
2,24.17
3,29.71
4,20.12
5,27.55


In [95]:
%%sql
SELECT * FROM products 
WHERE price <= 20
LIMIT 5;

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,price
1,18.48
6,14.9
12,14.98
13,14.37
14,14.77


In [97]:
%%sql
SELECT * FROM products
WHERE price >= 20
LIMIT 5;

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,price
0,21.78
2,24.17
3,29.71
4,20.12
5,27.55


### We can additionally combine $WHERE$ with $LIKE$ for pattern matching an $IN$ for membership 

####  For example we can find orders *Where the delivery_country begins with 'S'* using the $LIKE$ keyword with $WHERE$


#### This is similiar in pandas to the $str$.$contains$ method

#### A major difference between SQL $LIKE$ syntax and pandas $str$.$contains$ syntax is that pandas uses regular expression for pattern matching... 

#### *$Regular Expressions$ is a whole set of characters for indicating spaces, digits, and other characters and specifying the structure of various patterns in strings that we can search for*

#### Different SQL falvours have thier own pattern matching syntax, so one might have to look up how to write thr pattern matching syntax for the sql falvour being used.... as that for MySql differs from MSQL or Postgre SQL...

In [102]:
%%sql

SELECT id, delivery_country FROM orders
WHERE delivery_country like 'S%'
LIMIT 5;

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,delivery_country
2,Spain
11,Senegal
16,Sweden
16,Sweden
16,Sweden


#### Note: The *%* sign in *like 'S%'* refers to any number of letters or combination following *'S'*

In [92]:
%%sql

SELECT * FROM orders
WHERE customer_id IN (10, 200, 400);

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,customer_id,product_id,delivery_country
211,200,858,Madagascar
211,200,434,Mexico
542,400,80,Sweden
542,400,405,Sweden
818,10,778,Canada
818,10,60,Canada


#### Something all SQL falvours have in common is the $IN$ keyword for testing membership. 

#### This is similiar to *Pandas* $isin$ keyword, where we can check if a series or DataFrame  to see whether the values in a series are contained in some set of values.

#### *Set of values here could be an actual set or a list or a tuple or any kind of list-like container of values*

#### $Eg$
#### s = pd.Series (['Lion', 'Cat', 'Dog', 'Eagle', 'cow']), name = 'animal')
#### s.isin(['cow', 'lion])
#### It would return boolean values for each rows whether the row values matches any of the search values
#### 0 True
#### 1 False
#### 2 False
#### 3 False
#### 4 True

#### We can also combine them with the usual logical operators: $AND$, $OR$, and $NOT$

In [103]:
%%sql

SELECT * FROM orders
WHERE customer_id IN (10, 200, 400)
AND delivery_country NOT IN ('Madagascar', 'Canada');

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,customer_id,product_id,delivery_country
211,200,434,Mexico
542,400,80,Sweden
542,400,405,Sweden


In [111]:
%%sql

SELECT * FROM products
WHERE price < 10 OR price > 30
LIMIT 5;

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,price
10,31.74
17,7.27
22,33.03
41,8.98
44,5.8


## $Sorting$

#### To sort our values, we can *ORDER BY* one or more values. We can also choose whether we sort in ascendng $(ASC)$ or descending $(DESC)$ order. 

#### *SQL sorts in ascending $(ASC)$* order by default similiar to the python convention

In [117]:
%%sql

SELECT * FROM orders
ORDER BY customer_id
LIMIT 5;

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,customer_id,product_id,delivery_country
189,1,307,Canada
189,1,508,Poland
949,1,431,Canada
949,1,592,Canada
146,5,263,USA


#### We can $ORDER BY$ multiple columns and specify whether we want to ORDER in $(ASC)$ OR $(DESC)$ order

In [116]:
%%sql

SELECT * FROM orders
ORDER BY customer_id ASC, product_id DESC
LIMIT 5;

 * sqlite:///testdb.sqlite
   sqlite:///testdb1.sqlite
Done.


id,customer_id,product_id,delivery_country
949,1,592,Canada
189,1,508,Poland
949,1,431,Canada
189,1,307,Canada
146,5,864,USA
